In [1]:
!pip install lightgbm

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, log_loss
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [12]:
df = pd.read_csv("final_features.csv")
df = df.sample(n=90000, random_state=2)
print(df.shape)
df.tail()

(90000, 1190)


,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,...,86_q2_y,87_q2_y,88_q2_y,89_q2_y,90_q2_y,91_q2_y,92_q2_y,93_q2_y,94_q2_y,95_q2_y
34971,223062,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,17.199036,-27.237144,5.400709,28.044876,-30.063677,0.696971,39.786740,21.583338,-22.410946,5.302735
36569,121929,1,0.545450,0.499996,0.374995,0.299997,0.428569,0.333332,0.0,0.0,...,11.502255,0.144446,-3.402341,10.742955,-8.240209,3.727438,12.624058,9.990710,-2.459129,9.366579
30264,75717,0,0.499975,0.333322,0.499987,0.499987,0.499992,0.428565,0.0,0.0,...,14.411508,-10.531261,-11.455352,42.261574,-9.032182,27.496660,37.994663,25.499485,-21.955116,16.899961
9495,387155,1,0.749981,0.749981,0.999900,0.333322,0.799984,0.571420,0.0,1.0,...,-5.037388,-11.085898,-1.099428,-8.514374,-1.084490,-4.625764,5.879039,-5.895683,5.921881,7.653259
34432,77719,1,0.749981,0.749981,0.333322,0.199996,0.499994,0.399996,0.0,0.0,...,16.702414,-39.073620,1.129437,41.434350,17.061893,-11.556725,34.474068,40.167910,-44.360012,15.101011


In [13]:
print(df.columns)

Index(['id', 'is_duplicate', 'cwc_min', 'cwc_max', 'csc_min', 'csc_max',
       'ctc_min', 'ctc_max', 'last_word_eq', 'first_word_eq',
       ...
       '86_q2_y', '87_q2_y', '88_q2_y', '89_q2_y', '90_q2_y', '91_q2_y',
       '92_q2_y', '93_q2_y', '94_q2_y', '95_q2_y'],
      dtype='object', length=1190)


In [14]:
# Target
y = df['is_duplicate']  

# Features
X = df.drop(columns=['id','is_duplicate'], errors='ignore')


In [15]:
y.shape, X.shape

((90000,), (90000, 1188))

In [16]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=y)


In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((72000, 1188), (18000, 1188), (72000,), (18000,))

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on training data
rf.fit(X_train, y_train)

# Predict on test data
y_pred = rf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7122777777777778

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.79      0.78     11369
           1       0.62      0.58      0.60      6631

    accuracy                           0.71     18000
   macro avg       0.69      0.69      0.69     18000
weighted avg       0.71      0.71      0.71     18000



In [19]:
from sklearn.metrics import log_loss

# Predict probabilities for log loss
y_proba_rf = rf.predict_proba(X_test)

# Calculate log loss
loss_rf = log_loss(y_test, y_proba_rf)
print("Random Forest Log Loss:", loss_rf)


Random Forest Log Loss: 0.5056944155574885


In [20]:
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric="logloss" 
)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
y_pred_proba = xgb.predict_proba(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Log Loss:", log_loss(y_test, y_pred_proba))

Accuracy: 0.7406666666666667
F1 Score: 0.656259204712813
Log Loss: 0.47438708835725657


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, log_loss
from sklearn.model_selection import RandomizedSearchCV, train_test_split
rf = RandomForestClassifier(random_state=42)
param_dist = {
    'n_estimators': [100, 200, 300],         # keep small range
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}
rand_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,                 # reduce number of trials to save memory
    scoring="neg_log_loss",    # optimize log loss
    cv=3,                      # 3-fold CV
    verbose=2,
    random_state=42,
    n_jobs=1                   # use 1 job to avoid memory blow-up
)

rand_search.fit(X_train, y_train)
print("Best Params:", rand_search.best_params_)
print("Best CV Log Loss:", -rand_search.best_score_)


best_model = rand_search.best_estimator_

# Predictions
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Test Log Loss:", log_loss(y_test, y_pred_proba))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time= 2.4min
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time= 2.4min
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time= 2.4min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time= 2.6min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time= 2.6min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time= 2.6min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time= 4.2min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, 

c:\Users\SANHITH REDDY\.conda\envs\quora_env\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 1188 features, but RandomForestClassifier is expecting 138 features as input.

In [ ]:
lr = LogisticRegression(max_iter=2000, n_jobs=-1, solver="saga")
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
y_pred_proba = lr.predict_proba(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Log Loss:", log_loss(y_test, y_pred_proba))


C:\Users\SANHITH REDDY\.conda\envs\quora_env\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
lgb_model = lgb.LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

lgb_model.fit(X_train, y_train)

y_pred = lgb_model.predict(X_test)
y_pred_proba = lgb_model.predict_proba(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Log Loss:", log_loss(y_test, y_pred_proba))
